In [7]:
import ollama

# Inisialisasi Ollama client
ollama_client = ollama.Client()

def get_answer_stream(image_path, query, model_name="gemma3:12b"):
    """
    Mendapatkan jawaban dari model Ollama dengan mode streaming (teks muncul bertahap).
    """
    try:
        with open(image_path, "rb") as img_file:
            image_bytes = img_file.read()

        prompt = f"Query: {query}\nGambar: "
        
        # 1. Ubah stream=True
        response = ollama_client.generate(
            model=model_name,
            prompt=prompt,
            stream=True,  # Mengaktifkan mode streaming
            images=[image_bytes]
        )

        print(f"Query: {query}")
        print("Jawaban: ", end="", flush=True)

        # 2. Iterasi melalui chunks yang dikirim oleh Ollama
        for chunk in response:
            chunk_text = chunk['response']
            print(chunk_text, end="", flush=True)  # Print tanpa baris baru dan paksa keluar ke terminal
            
        print() # Tambahkan baris baru di akhir jawaban

    except Exception as e:
        print(f"\nError saat berinteraksi dengan Ollama: {e}")

# Contoh penggunaan
if __name__ == "__main__":
    path = "c:\\Users\\Ilmu Komputer\\Downloads\\image.png"
    q = "rubahlah table menjadi tag html murni lalu berikan judul yang tepat. Jangan ada kalimat pembuka atau penutup"
    
    get_answer_stream(path, q)

Query: rubahlah table menjadi tag html murni lalu berikan judul yang tepat. Jangan ada kalimat pembuka atau penutup
Jawaban: 

```html
<h1>Daftar Informasi Keterbukaan Universitas Pendidikan Ganesha Tahun 2024</h1>

<table>
  <thead>
    <tr>
      <th>Nama</th>
      <th>NIM</th>
      <th>Prodi</th>
      <th>Fakultas</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>I Gusti Ayu Gita Dewi Maheswari</td>
      <td>2115051054</td>
      <td>Pendidikan Teknik Informatika</td>
      <td>FTK</td>
    </tr>
    <tr>
      <td>Wayan Restu Cahyana</td>
      <td>2215101035</td>
      <td>Ilmu Komputer</td>
      <td>FTK</td>
    </tr>
    <tr>
      <td>I Putu Duta Mahardika</td>
      <td>2118011045</td>
      <td>Kedokteran</td>
      <td>FK</td>
    </tr>
    <tr>
      <td>Ketut Lingga Amritiya</td>
      <td>2218011019</td>
      <td>Kedokteran</td>
      <td>FK</td>
    </tr>
    <tr>
      <td>Putu Vina Aryadnyani</td>
      <td>211031135</td>
      <td>PGSD</td>
      <td>FIP</td>
    </tr>
    <tr>
      <td>Kadek Adrian Surya Indra Wirawan</td>
      <td>2211031128</td>
      <td>PGSD</td>
      <td>F

Ollama versi stream

In [ ]:
import os
import base64
import asyncio
import ollama
import nest_asyncio
from io import BytesIO
from PIL import Image
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, EasyOcrOptions
from docling.document_converter import PdfFormatOption

# 1. FIX: Menangani error "asyncio.run() cannot be called from a running event loop"
nest_asyncio.apply()

# 2. KONFIGURASI PIPELINE DOCLING DENGAN EASYOCR
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.generate_picture_images = True

# PENTING: Gunakan EasyOCR dengan force_full_page_ocr
pipeline_options.ocr_options = EasyOcrOptions(
    force_full_page_ocr=True,  # Paksa OCR di seluruh halaman
    lang=["en", "id"]  # Bahasa: English dan Indonesia
)

converter = DocumentConverter(
    format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
)

# 3. HELPER: Konversi Gambar ke Base64
def img_to_b64(pil_img):
    """
    Mengonversi PIL Image ke string Base64 untuk dikirim ke VLM.
    """
    buf = BytesIO()
    pil_img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# 4. HELPER: Komunikasi dengan Gemma 3 via Ollama
async def call_gemma_vision(prompt, image_b64=None):
    """Fungsi wrapper untuk mengirim gambar ke Gemma 3"""
    messages = [{'role': 'user', 'content': prompt, 'images': [image_b64]}]
    
    try:
        response = ollama.chat(model='gemma3:12b', messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"Error: Gagal memproses gambar ({str(e)})"

# 5. FUNGSI UTAMA: Konversi PDF ke Markdown dengan Deskripsi Gambar
async def generate_multimodal_markdown(pdf_path, output_path):
    print(f"--- Memulai Konversi: {os.path.basename(pdf_path)} ---")
    print(f"--- Menggunakan EasyOCR dengan force_full_page_ocr=True ---")
    
    if not os.path.exists(pdf_path):
        print(f"Error: File {pdf_path} tidak ditemukan.")
        return

    # Proses konversi dengan Docling
    result = converter.convert(pdf_path)
    doc = result.document
    
    md_content = []
    content_found = False  # Track apakah ada konten yang berhasil diekstrak

    # Iterasi setiap elemen dokumen hasil layout analysis
    for element, _ in doc.iterate_items():
        label = element.label.lower()
        
        # A. Penanganan Judul & Header
        if label in ["heading", "section_header", "title"]:
            level = getattr(element, 'level', 1)
            level = max(1, min(level, 6))
            heading_marker = "#" * level
            text = element.text.strip()
            if text:
                md_content.append(f"\n{heading_marker} {text}\n")
                content_found = True
            
        # B. Penanganan Teks Biasa & List
        elif label in ["paragraph", "text", "list_item", "caption"]:
            text = element.text.strip()
            if text:
                md_content.append(text)
                content_found = True

        # C. Penanganan Tabel - LANGSUNG DARI DOCLING
        elif label == "table":
            print("   > Memproses Tabel (Docling)...")
            try:
                table_md = element.export_to_markdown()
                if table_md and table_md.strip():
                    md_content.append(f"\n{table_md}\n")
                    content_found = True
            except Exception as e:
                print(f"   ! Error memproses tabel: {str(e)}")

        # D. Penanganan Gambar - FOKUS DESKRIPSI DENGAN GEMMA
        elif label in ["picture", "figure", "graphic"]:
            if hasattr(element, 'image') and element.image:
                print(f"   > Menganalisis Gambar dengan Gemma...")
                
                try:
                    img_pil = element.image.pil_image
                    b64_data = img_to_b64(img_pil)
                    
                    # Cek apakah ada konten teks yang berhasil diekstrak sebelumnya
                    # Jika tidak, kemungkinan ini gambar yang gagal di-OCR
                    recent_text_length = sum(len(c) for c in md_content[-3:]) if len(md_content) >= 3 else 0
                    
                    if recent_text_length < 50:  # Sedikit atau tidak ada teks nearby
                        print("      > Docling OCR mungkin gagal, gunakan Gemma untuk ekstraksi teks penuh...")
                        # Prompt untuk ekstraksi teks + deskripsi
                        prompt = """Analisis gambar ini secara menyeluruh dan berikan output dengan format berikut:

                        1. Ekstrak SEMUA teks yang terlihat dalam gambar (judul, paragraf, nomor, bullet points, dll)
                        2. Setelah itu, jelaskan gambar dengan awalan "Gambar ini adalah..." yang mencakup:
                        - Jenis gambar (screenshot, dokumen, diagram, foto, dll)
                        - Elemen visual penting
                        - Layout atau struktur

                        Format output:
                        [Teks yang diekstrak jika ada]

                        Gambar ini adalah [penjelasan detail]..."""
                    else:
                        # Ada teks nearby, hanya perlu deskripsi visual
                        prompt = """Jelaskan gambar ini dengan detail untuk sistem RAG. 
                        Mulai penjelasan dengan "Gambar ini adalah..." dan berikan deskripsi yang lengkap mencakup:
                        - Jenis/kategori gambar (screenshot, diagram, foto, logo, tanda tangan, ilustrasi, dll)
                        - Konten utama yang terlihat
                        - Teks yang ada di dalam gambar (jika ada)
                        - Detail penting lainnya

                        Berikan deskripsi dalam 3-5 kalimat yang informatif."""
                    
                    description = await call_gemma_vision(prompt, image_b64=b64_data)
                    md_content.append(f"\n{description.strip()}\n")
                    content_found = True
                    
                except Exception as e:
                    print(f"   ! Error memproses gambar: {str(e)}")
            else:
                print(f"   ! Warning: Elemen {label} tanpa data gambar.")

    # 6. PENANGANAN PDF KOSONG atau GAGAL EKSTRAKSI
    if not content_found or len(md_content) == 0:
        print("\n   ! WARNING: Tidak ada konten yang berhasil diekstrak dari PDF!")
        print("   ! Kemungkinan: PDF kosong, OCR gagal total, atau format tidak didukung.")
        md_content.append("<!-- Konten PDF tidak dapat diekstrak. Kemungkinan PDF kosong atau format tidak didukung. -->")

    # 7. PENYIMPANAN KE FILE MARKDOWN
    final_md = "\n\n".join(md_content)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_md)
    
    print(f"\n--- SELESAI! ---")
    print(f"File Markdown Multimodal disimpan di: {output_path}")
    if not content_found:
        print(f"⚠️  PERHATIAN: File output mungkin kosong atau minimal karena OCR gagal.")


# Eksekusi
pdf_input = "C:\\Users\\Ilmu Komputer\\OneDrive\\Desktop\\portofolio\\RAG\\defi-rag-agent\\data\\input_pdfs\\SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.pdf"
md_output = "C:\\Users\\Ilmu Komputer\\OneDrive\\Desktop\\portofolio\\RAG\\defi-rag-agent\\data\\output_markdowns\\SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.md"
await generate_multimodal_markdown(pdf_input, md_output)

2026-01-14 23:53:14,085 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-14 23:53:14,126 - INFO - Going to convert document batch...
2026-01-14 23:53:14,126 - INFO - Initializing pipeline for StandardPdfPipeline with options hash ca9cc13e2c0b26e715e00cb43fd00047
2026-01-14 23:53:14,126 - INFO - Accelerator device: 'cuda:0'


--- Memulai Konversi: SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.pdf ---
--- Menggunakan EasyOCR dengan force_full_page_ocr=True ---


2026-01-14 23:53:16,346 - INFO - Accelerator device: 'cuda:0'
2026-01-14 23:53:18,088 - INFO - Accelerator device: 'cuda:0'
2026-01-14 23:53:18,934 - INFO - Processing document SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.pdf
2026-01-14 23:53:54,999 - INFO - Finished converting document SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.pdf in 41.01 sec.


   > Menganalisis Gambar dengan Gemma...
      > Docling OCR mungkin gagal, gunakan Gemma untuk ekstraksi teks penuh...


2026-01-14 23:54:34,882 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2026-01-14 23:54:34,885 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.


   > Memproses Tabel (Docling)...
   > Menganalisis Gambar dengan Gemma...


2026-01-14 23:54:58,169 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2026-01-14 23:54:58,171 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,174 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,177 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,179 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,182 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,183 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,184 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
2026-01-14 23:54:58,190 - WARNING - Usage of TableItem.export_to_markdown() without `doc` argument i

   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Memproses Tabel (Docling)...
   > Menganalisis Gambar dengan Gemma...


2026-01-14 23:55:27,330 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



--- SELESAI! ---
File Markdown Multimodal disimpan di: C:\Users\Ilmu Komputer\OneDrive\Desktop\portofolio\RAG\defi-rag-agent\data\output_markdowns\SK-Daftar-Informasi-Informasi-Publik-Undiksha-Tahun-2024.md
